## Markov Filter

#### Table of Contents

* [Introduction to the Markov Filter](#first-bullet)

* [Application of the Markov Filter](#second-bullet)

* [Managing problems: multiple times the same value + ](#third-bullet)

* [Visualization with heatmap](#forth-bullet)




#### Introduction to the Markov Filter <a class="anchor" id="IntroductionMarkovFilter"></a>

To get a better accuracy of the position of the robot, a Markov Localisation Filter is used. It is based on the Bayes Filter.
The Markov process consists of a sequence of states, where each future state only depends on the current state.
Markov Localization is particularly useful in scenarios where the environment is partially observable, and there is uncertainty in both the robot's actions and sensor measurements.

The filtering is done after each segment of the global path that the robot has driven

This is the header of the Markov function:
- markov(x_robot,y_robot,x_CV,y_CV,confindence_CV,first_step)

Input: 
- belief position of the robot based on the global path (x_robot,y_robot)
- position of the robot given by CV (x_CV,y_CV)
- confidence of the robot's position given by CV
- first_step: True if it the robot has just driven the first segment of the global path, False otherwise

Output:
- more accurate position of the robot (x,y)


Source: Class on MOBILE ROBOTS 7/8- Uncertainties from Prof. Mondada of the class "Basics of Mobile Robotics", MICRO-452.



 
#### Implementation of the Markov filter <a class="anchor" id="ImplementationMarkovFilter"></a>

The algorithm consists of the following steps:
0) Initialize the maps
1) Belief state
2) Sensor update
3) Motion update
4) Calculation step
4) Recursive estimation


Decisions taken:
- The size of the camera frame is: 480x640 - hight x width. We chose the map to have the size of 75 x 100 which has the same ratio as the camera frame.



0) Initializing the maps
This step is done once at the start of the program. To compute the Markov filter, we need three maps of the same size (75x100) "pixels". Please note that "pixels" refers to the individual squares on the map, not the camera pixels.
The base map (map) is inisialized with a base probability of 1/(width*height) of the map.
The map_estimate_robot and the map_estimate_CV containing the belief and measured position of the robot respecitively, are empty.


# map: each square has the prob = 1/(height * length)
# map_estimate_robot: empty map with size height x length
# map_estimate_CV: empty map with size height x length
def initialize_maps(height,length):
    global map, map_estimate_robot, map_estimate_CV
    
    # initialize map
    initial_prob = 1/(length*height)
    map = np.empty((height,length), float)
    map.fill(initial_prob)

    # create empty map_estimate_robot
    map_estimate_robot = np.empty((height,length), float)
    map_estimate_robot.fill(0.0)

    # create empty map_estimate_CV
    map_estimate_CV = np.empty((height,length), float)
    map_estimate_CV.fill(0.0)


1) The belief state
In our case, the belief state corresponds to the position the robot is supposed to have after having driven a segment according to the global path. It is therefore simply equal to the current step of the global path.
The probability that the robot is in the "pixel" it is supposed to be in is 0.8. The probability that it's in one of the surrounding pixels is 0.8/8 (there are 8 surrounding pixels).

 !!!!!!!!!!!! DID I NOT IMPLEMENT THIS !!!!!!!!!!!!!!!
To make sure that this also works at the edges of the map,  ??????????????????

2) Sensor update
The CV gives the position of the robot and also a confidence value. The given position has a confindence of that confidence value, the surrounding "pixels" have a probability of (1-confidence)/8 (there are 8 surrounding pixels).

3) Calculation step
The map, map_estimate_CV, and map_estimate_robot are multiplied and subsequently normalized so that the sum of all the elements in the map is equal to 1. 

Filtered position -> gives the correct position, if there's multiple values that have the same prob -> it takes the "pixel" that is closest to the position estimated by the robot




4) There is no need for a motion update as the global path is simply recalculated based on the position given by the filter after each segment.


5) Recursive estimation
The filtering is done at every step.


## Limitations of the Markov filter <a class="anchor" id="LimitationsMarkovFilter"></a>




- too many pixels -> too little weight, could have chosen bigger ones but then u loose precision






##### things to write about
- Why this filter?
In this project, we have some uncertainty in the sensor (camera) and some in the movement of the robot. 


Markov Localization is particularly  useful in scenarios where the environment is partially observable, and there is uncertainty in both the robot's actions and sensor measurements. It has applications in mobile robotics, autonomous vehicles, and other fields where tracking and localization are essential. The most well-known algorithm for Markov Localization is the Monte Carlo Localization (MCL) algorithm, also known as Particle Filter Localizatio

-  Why 


In [1]:
# MARKOV

# Imports
import numpy as np
import math

import matplotlib.pyplot as plt
import seaborn as sn 

import tdmclient.notebook


In [ ]:


def display_heatmap(data_to_display):
    #hm = sn.heatmap(data = data_to_display, annot=False, linewidth=.5, vmin=0, vmax=1.0) 
    hm = sn.heatmap(data = data_to_display, annot=False, linewidth=.5, vmin=0, vmax=1.0) 
    hm.invert_yaxis()
    
def neighbouring_pixels(x,y,prob,any_map):
    global map_estimate_robot
 
    any_map[x-1][y-1] = prob
    any_map[x-1][y] = prob
    any_map[x-1][y+1] = prob

    any_map[x][y-1] = prob
    any_map[x][y+1] = prob

    any_map[x+1][y-1] = prob
    any_map[x+1][y] = prob
    any_map[x+1][y+1] = prob

 
def initialize_maps(length,height):
    global map, map_estimate_robot, map_estimate_CV
    
    # initialize map
    start_prob = 1/(length*height)
    map = np.empty((length,height), float)
    map.fill(start_prob)
    #display_heatmap(map)

    # initialize map_estimate_robot
    map_estimate_robot = np.empty((length,height), float)
    map_estimate_robot.fill(0.0)

    # initialize map_estimate_CV
    map_estimate_CV = np.empty((length,height), float)
    map_estimate_CV.fill(0)

    print(map)


def estimate_robot(x,y):
    global map_estimate_robot   
    
    # reset map to 0
    map_estimate_robot.fill(0)

    # add prob
    P_HIT = 0.8
    P_MISS = 0.2/8 #8 neighboring pixels
    map_estimate_robot[x][y] = P_HIT
    neighbouring_pixels(x,y,P_MISS,map_estimate_robot)
    
    #display_heatmap(map_estimate_robot)
    print(map_estimate_robot)


def estimate_CV(x,y,confidence_CV):
    # reset map to 0
    map_estimate_CV.fill(0)

    # add prob
    MEAS_P_HIT = confidence_CV
    MEAS_P_MISS = (1-confidence_CV)/8
    map_estimate_CV[x][y] = MEAS_P_HIT
    neighbouring_pixels(x,y,MEAS_P_MISS,map_estimate_CV)
    
    #display_heatmap(map_estimate_CV)
    print(map_estimate_CV)

def multiply_maps():
    global map, map_estimate_CV, map_estimate_robot
    temp = np.multiply(map_estimate_CV,map_estimate_robot)
    
    # if the estimated position of the robot and by CV are too far apart, or the last position of the robot and its position now, the whole map after multiplication is = 0
    # in this case, set map = map_estimate_robot as it is less likely that the position measured by the robot is off by a lot than the CV
    if (not np.any(temp)):
        map = map
    else:
        map = np.multiply(map,temp)

    if (not np.any(map)):
        map = map_estimate_CV  # if start map and new position is too different, just take the position of the CV (e.g. displacing robot by hand)

    print("multiply", map)

# Function to normalize the values in map such that the sum of all elements is 1
def normalize_map():
    global map
  
    # Calculate the sum of all values in the map array
    sum = np.sum(map)
     # Divide each element in the map array by the sum to normalize the values
    map = np.divide(map,sum)
    
 


def multiple_highest_prob_with_same_value(pos_robot,indices):

    # save estimated position of robot, estimated by itself
    pos_robot_x = pos_robot[0].item()
    pos_robot_y = pos_robot[1].item()
    point = np.array([pos_robot_x, pos_robot_y])

    # put values of pos with the same highest prob in a table, table[0] -> x, table[1] -> 1
    table = np.array(indices).tolist()

    # Transpose the table array
    table_array = np.array(table).T

    # Calculate Euclidean distances
    distances = np.linalg.norm(table_array - point, axis=1)

    # Find the index of the minimum distance
    closest_index = np.argmin(distances)
    X_ROBOT = table[0][1]
    Y_ROBOT = table[1][1]

    return X_ROBOT,Y_ROBOT


def filtered_pos_robot():
    # Find indices where the maximum value occurs in map
    indices = np.where(map == map.max())
    # Find indices where the maximum value occurs in map_estimate_robot
    pos_robot = np.where(map_estimate_robot == map_estimate_robot.max())
    
    # Check if there is a single maximum value in map
    if (np.size(indices)==2): 
        # If there is only one maximum, assign X_ROBOT and Y_ROBOT directly
        X_ROBOT = indices[0].item()
        Y_ROBOT = indices[1].item()
    else:
        # If there are multiple maxima, use a function to resolve the conflict
        X_ROBOT, Y_ROBOT = multiple_highest_prob_with_same_value(pos_robot,indices)
    
    # Return the X and Y coordinates of the robot
    return X_ROBOT,Y_ROBOT


def markov(x_robot,y_robot,x_CV,y_CV,confindence_CV):
    # Initialize local variables
    x_filtered_pos_robot = 0
    y_filtered_pos_robot = 0
    # take integer value (rounded down)
    x_robot = np.int64(x_robot)
    y_robot = np.int64(y_robot)
    x_CV = np.int64(x_CV)
    y_CV = np.int64(y_CV)
    
    
    estimate_robot(x_robot,y_robot)
    estimate_CV(x_CV,y_CV,confindence_CV)
    multiply_maps()
    normalize_map()
    x_filtered_pos_robot, y_filtered_pos_robot = filtered_pos_robot()

    return x_filtered_pos_robot, y_filtered_pos_robot
